rodzaje jednostek

machiny oblężnicze: taran, katapulta, wieża oblężnicza, balista

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
conn = sqlite3.connect('constants.db')
cursor = conn.cursor()

Print tables names

In [35]:
# Wykonanie zapytania SQL, aby pobrać nazwy wszystkich tabel
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Pobranie wyników zapytania
tables = cursor.fetchall()

# Wyświetlenie nazw tabel
for table in tables:
    print(table[0])


Units
sqlite_sequence
Buildings
Town_Hall
Barracks
Stable
Workshop
Forge
Market
Military_Square
Palace
Sawmill
Brickyard
Ironworks
Granary
Warehouse
Hideout
Wall
Church
Watchtower


# Units table

Print tables content

In [36]:
df = pd.read_sql_query("SELECT * FROM sqlite_sequence", conn)
df

,name,seq
0,Buildings,17


Adding cols to table

In [23]:
query = '''ALTER TABLE Units
ADD COLUMN Description TEXT;'''


cursor.execute(query)
conn.commit()

Adding and filling rows in table

In [12]:

name = 'Siege Tower'
wood = 1000
clay = 600
iron = 600
population = 12
speed = 30           # minutes
loot_capacity = 0
attack_infantry = 50
attack_cavalry = 50
defence = 200
defence_ranged = 120
defence_cavalry = 100

query = '''INSERT INTO Units (Name, Wood, Clay, Iron, Population, Speed, Loot_Capacity, Attack_Infantry, 
Attack_Cavalry, Defence, Defence_Ranged, Defence_Cavalry) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
data = (name, wood, clay, iron, population, speed, loot_capacity, attack_infantry, attack_cavalry, defence, defence_ranged, defence_cavalry)

cursor.execute(query, data)
conn.commit()

df = pd.read_sql_query("SELECT * FROM Units", conn)
df

,id,Name,Wood,Clay,Iron,Population,Speed,Loot_Capacity,Attack_Infantry,Attack_Cavalry,Defence,Defence_Ranged,Defence_Cavalry
0,1,Pikemen,50,30,10,1,18,25,10,40,15,10,45
1,2,Swordsmen,10,30,50,1,22,20,20,20,35,25,20
2,3,Axemen,30,30,40,1,18,10,40,10,15,5,10
3,4,Archer,80,30,40,1,18,15,45,35,5,5,5
4,5,Saboteur,30,50,20,1,15,35,10,5,10,5,5
5,6,Halberdier,70,40,40,2,22,20,40,45,25,15,55
6,7,Crossbowman,100,40,60,2,22,10,55,35,15,10,5
7,8,Legionnaire,30,60,120,2,25,20,35,30,50,40,25
8,9,Scout,30,70,20,2,8,0,0,0,2,2,2
9,10,Light Cavalry,125,150,200,4,10,90,130,90,40,25,30


In [66]:

unit_id = 1
fill_value = 10

# Zapytanie SQL do aktualizacji wartości
query = '''UPDATE Units 
           SET Defence_Ranged=?
           WHERE id = ?'''

# Wykonanie zapytania SQL
cursor.execute(query, (fill_value, unit_id))

# Zatwierdzenie zmian w bazie danych
conn.commit()

In [10]:
query = '''DELETE FROM Units 
           WHERE Name = ?'''

# Wartość, którą chcemy usunąć
name_to_delete = 'Siege Tower'

# Wykonanie zapytania SQL
cursor.execute(query, (name_to_delete,))

# Zatwierdzenie zmian w bazie danych
conn.commit()

# Buildings

## Central table
Create, add and modify central tables content. Central table store information aobut buildings and their description

In [37]:
cursor.execute("CREATE TABLE Buildings(Building_id INTEGER PRIMARY KEY AUTOINCREMENT, Name TEXT NOT NULL, Description TEXT);")
conn.commit()


In [ ]:
name = "Watchtower"
description = "The watchtower supports the village's defense by providing the player with valuable information about incoming attacks."

query = '''INSERT INTO Buildings (Name, Description)
VALUES (?, ?)'''
data = (name, description)

cursor.execute(query, data)
conn.commit()

df = pd.read_sql_query("SELECT * FROM Buildings", conn)
df

In [168]:
name = 'Barracks'
df = pd.read_sql_query(f"SELECT * FROM Buildings WHERE Name=='{name}'", conn)
df

,Building_id,Name,Description
0,2,Barracks,"In the barracks, you can recruit infantry. The..."


## Direct add/modify buildings
Get list of buildings from Buildings table. Star-structured tables are created based on this data. The tables will include building levels, their costs and effects.

In [28]:
_buildings = pd.read_sql_query("SELECT Name FROM Buildings", conn)
_buildings


,Name
0,Town Hall
1,Barracks
2,Stable
3,Workshop
4,Forge
5,Market
6,Military Square
7,Palace
8,Sawmill
9,Brickyard


In [27]:
for idx, row in _buildings.iterrows():
    table_name = row['Name'].replace(' ', '_')
    query = f'''
    CREATE TABLE {table_name} (
        Level INTEGER PRIMARY KEY,
        Wood INTEGER,
        Clay INTEGER,
        Iron INTEGER,
        Population INTEGER,
        Coefficient FLOAT,
        Building_id TEXT,
        FOREIGN KEY (Building_id) REFERENCES Buildings(Name)
    );'''
    
    cursor.execute(query)
    conn.commit()


### Town Hall

In [83]:
for level in range(1, 31):
    query = '''INSERT INTO Town_Hall (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(90 * np.exp(0.232*(level-1)))
    clay = int(80 * np.exp(0.244*(level-1)))
    iron = int(70 * np.exp(0.226*(level-1)))
    population = int(np.exp(0.148*(level-1)))
    coef = round(1 - 0.65 * np.log10(0.5*(level+1)), 2)
    building_name = "Town Hall"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

In [99]:
df = pd.read_sql_query("SELECT * FROM Town_Hall", conn)
df

,Level,Wood,Clay,Iron,Population,Coefficient,Building_id
0,1,90,80,70,1,1.00,Town Hall
1,2,113,102,87,1,0.89,Town Hall
2,3,143,130,110,1,0.80,Town Hall
3,4,180,166,137,1,0.74,Town Hall
4,5,227,212,172,1,0.69,Town Hall
5,6,287,270,216,2,0.65,Town Hall
6,7,362,345,271,2,0.61,Town Hall
7,8,456,441,340,2,0.58,Town Hall
8,9,575,563,426,3,0.55,Town Hall
9,10,726,719,535,3,0.52,Town Hall


### Barracks

In [ ]:
for level in range(1, 26):
    wood = int(200 * np.exp(0.23*(level-1)))
    clay = int(170 * np.exp(0.247*(level-1)))
    iron = int(90 * np.exp(0.231*(level-1)))
    population = int(np.exp(0.16*(level-1)))
    coef = round(1 - 0.77 * np.log10(0.5*(level+1)), 2)
    building_name = "Barracks"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''UPDATE Barracks SET Population=10 WHERE Level=1'''
cursor.execute(query)
conn.commit()

In [104]:
df = pd.read_sql_query("SELECT * FROM Barracks", conn)
df

,Level,Wood,Clay,Iron,Population,Coefficient,Building_id
0,1,200,170,90,10,1.00,Barracks
1,2,251,217,113,1,0.86,Barracks
2,3,316,278,142,1,0.77,Barracks
3,4,398,356,179,1,0.69,Barracks
4,5,501,456,226,1,0.63,Barracks
5,6,631,584,285,2,0.58,Barracks
6,7,794,748,359,2,0.54,Barracks
7,8,1000,957,453,3,0.50,Barracks
8,9,1259,1226,571,3,0.46,Barracks
9,10,1584,1569,719,4,0.43,Barracks


### Stable

In [37]:
for level in range(1, 26):
    query = '''INSERT INTO Stable (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(270 * np.exp(0.234*(level-1)))
    clay = int(240 * np.exp(0.25*(level-1)))
    iron = int(160 * np.exp(0.252*(level-1)))
    population = int(np.exp(0.17*(level-1)))
    coef = round(1 - 0.72 * np.log10(0.5*(level+1)), 2)
    building_name = "Stable"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''UPDATE Stable SET Population=12 WHERE Level=1'''
cursor.execute(query)
conn.commit()

In [110]:
df = pd.read_sql_query("SELECT * FROM Stable", conn)
df

,Level,Wood,Clay,Iron,Population,Coefficient,Building_id
0,1,270,240,160,12,1.00,Stable
1,2,341,308,205,1,0.87,Stable
2,3,431,395,264,1,0.78,Stable
3,4,544,508,340,1,0.71,Stable
4,5,688,652,438,1,0.66,Stable
5,6,869,837,564,2,0.61,Stable
6,7,1099,1075,725,2,0.57,Stable
7,8,1389,1381,933,3,0.53,Stable
8,9,1755,1773,1201,3,0.50,Stable
9,10,2218,2277,1545,4,0.47,Stable


### Workshop

In [42]:
for level in range(1, 16):
    query = '''INSERT INTO Workshop (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(300 * np.exp(0.226*(level-1)))
    clay = int(240 * np.exp(0.251*(level-1)))
    iron = int(260 * np.exp(0.225*(level-1)))
    population = int(np.exp(0.16*(level-1)))
    coef = round(1 - 0.65 * np.log10(0.5*(level+1)), 2)
    building_name = "Workshop"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''UPDATE Workshop SET Population=8 WHERE Level=1'''
cursor.execute(query)
conn.commit()

In [111]:
df = pd.read_sql_query("SELECT * FROM Workshop", conn)
df

,Level,Wood,Clay,Iron,Population,Coefficient,Building_id
0,1,300,240,260,8,1.00,Workshop
1,2,376,308,325,1,0.89,Workshop
2,3,471,396,407,1,0.80,Workshop
3,4,590,509,510,1,0.74,Workshop
4,5,740,655,639,1,0.69,Workshop
5,6,928,841,800,2,0.65,Workshop
6,7,1164,1082,1002,2,0.61,Workshop
7,8,1459,1390,1255,3,0.58,Workshop
8,9,1829,1787,1572,3,0.55,Workshop
9,10,2293,2297,1969,4,0.52,Workshop


### Forge

In [45]:
for level in range(1, 21):
    query = '''INSERT INTO Forge (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(220 * np.exp(0.232*(level-1)))
    clay = int(180 * np.exp(0.241*(level-1)))
    iron = int(240 * np.exp(0.235*(level-1)))
    population = int(2 * np.exp(0.18*(level-1)))
    coef = round(1 - 0.75 * np.log10(0.5*(level+1)), 2)
    building_name = "Forge"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''UPDATE Forge SET Population=20 WHERE Level=1'''
cursor.execute(query)
conn.commit()

In [114]:
df = pd.read_sql_query("SELECT * FROM Forge", conn)
df

,Level,Wood,Clay,Iron,Population,Coefficient,Building_id
0,1,220,180,240,20,1.00,Forge
1,2,277,229,303,2,0.87,Forge
2,3,349,291,383,2,0.77,Forge
3,4,441,370,485,3,0.70,Forge
4,5,556,471,614,4,0.64,Forge
5,6,701,600,777,4,0.59,Forge
6,7,885,764,983,5,0.55,Forge
7,8,1116,972,1243,7,0.51,Forge
8,9,1407,1237,1572,8,0.48,Forge
9,10,1775,1574,1989,10,0.44,Forge


### Market

In [47]:
for level in range(1, 26):
    query = '''INSERT INTO Market (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(100 * np.exp(0.232*(level-1)))
    clay = int(100 * np.exp(0.245*(level-1)))
    iron = int(100 * np.exp(0.232*(level-1)))
    population = int(1 * np.exp(0.205*(level-1)))
    coef = int(np.exp(0.12*(level-1)))
    building_name = "Market"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''UPDATE Market SET Population=20 WHERE Level=1'''
cursor.execute(query)
conn.commit()

query = '''TABLE Market RENAME COLUMN Coefficint TO Merchants'''
cursor.execute(query)
conn.commit()

In [116]:
df = pd.read_sql_query("SELECT * FROM Market", conn)
df

,Level,Wood,Clay,Iron,Population,Merchants,Building_id
0,1,100,100,100,20,1.0,Market
1,2,126,127,126,1,1.0,Market
2,3,159,163,159,1,1.0,Market
3,4,200,208,200,1,1.0,Market
4,5,252,266,252,2,1.0,Market
5,6,318,340,318,2,1.0,Market
6,7,402,434,402,3,2.0,Market
7,8,507,555,507,4,2.0,Market
8,9,639,709,639,5,2.0,Market
9,10,806,907,806,6,2.0,Market


### Military Square

In [64]:
query = '''INSERT INTO 'Military_Square' (Level, Wood, Clay, Iron, Population, Building_id) VALUES (?,?,?,?,?,?)'''
building_name = "Miliatry Square"
cursor.execute(query, (1, 10, 40, 20, 0, building_name))
conn.commit()

query = '''ALTER TABLE 'Military_Square' DROP COLUMN Coefficient;'''
cursor.execute(query)
conn.commit()

In [76]:
df = pd.read_sql_query("SELECT * FROM 'Military_Square'", conn)
df

,Level,Wood,Clay,Iron,Population,Building_id
0,1,10,40,20,0,Miliatry Square


### Palace

In [73]:
query = '''INSERT INTO Palace (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
building_name = "Palace"
cursor.execute(query, (3, 65000, 100000, 50000, 25, 0.75, building_name))
conn.commit()

In [75]:
df = pd.read_sql_query("SELECT * FROM Palace", conn)
df

,Level,Wood,Clay,Iron,Population,Coefficient,Building_id
0,1,15000,25000,10000,80,1.00,Palace
1,2,30000,50000,20000,15,0.90,Palace
2,3,65000,100000,50000,25,0.75,Palace


### Sawmill

In [95]:
for level in range(1, 31):
    query = '''INSERT INTO Sawmill (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(50 * np.exp(0.215*(level-1)))
    clay = int(50 * np.exp(0.236*(level-1)))
    iron = int(40 * np.exp(0.211*(level-1)))
    population = int(np.exp(0.125*(level-1)))
    coef = int(30 * np.exp(0.149*(level-1)) + 5 * (level - 1))
    building_name = "Sawmill"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Sawmill RENAME COLUMN Coefficient TO Production;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Sawmill SET Population=5 WHERE Level=1;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Sawmill SET Production=2400 WHERE Level=30;'''
cursor.execute(query)
conn.commit()

In [96]:
df = pd.read_sql_query("SELECT * FROM Sawmill", conn)
df

,Level,Wood,Clay,Iron,Population,Production,Building_id
0,1,50,50,40,1,30.0,Sawmill
1,2,61,63,49,1,39.0,Sawmill
2,3,76,80,61,1,50.0,Sawmill
3,4,95,101,75,1,61.0,Sawmill
4,5,118,128,93,1,74.0,Sawmill
5,6,146,162,114,1,88.0,Sawmill
6,7,181,206,141,2,103.0,Sawmill
7,8,225,260,175,2,120.0,Sawmill
8,9,279,330,216,2,138.0,Sawmill
9,10,346,418,267,3,159.0,Sawmill


### Brickyard

In [118]:
for level in range(1, 31):
    query = '''INSERT INTO Brickyard (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(65 * np.exp(0.24*(level-1)))
    clay = int(50 * np.exp(0.236*(level-1)))
    iron = int(40 * np.exp(0.213*(level-1)))
    population = int(np.exp(0.137*(level-1)))
    coef = int(30 * np.exp(0.149*(level-1)) + 5 * (level - 1))
    building_name = "Brickyard"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Brickyard RENAME COLUMN Coefficient TO Production;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Brickyard SET Population=10 WHERE Level=1;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Brickyard SET Production=2400 WHERE Level=30;'''
cursor.execute(query)
conn.commit()

In [122]:
df = pd.read_sql_query("SELECT * FROM Brickyard", conn)
df

,Level,Wood,Clay,Iron,Population,Production,Building_id
0,1,65,50,40,1,30.0,Brickyard
1,2,82,63,49,1,39.0,Brickyard
2,3,105,80,61,1,50.0,Brickyard
3,4,133,101,75,1,61.0,Brickyard
4,5,169,128,93,1,74.0,Brickyard
5,6,215,162,116,1,88.0,Brickyard
6,7,274,206,143,2,103.0,Brickyard
7,8,348,260,177,2,120.0,Brickyard
8,9,443,330,219,2,138.0,Brickyard
9,10,563,418,272,3,159.0,Brickyard


### Ironworks

In [123]:
for level in range(1, 31):
    query = '''INSERT INTO Ironworks (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(75 * np.exp(0.225*(level-1)))
    clay = int(65 * np.exp(0.243*(level-1)))
    iron = int(70 * np.exp(0.213*(level-1)))
    population = int(2 * np.exp(0.146*(level-1)))
    coef = int(30 * np.exp(0.149*(level-1)) + 5 * (level - 1))
    building_name = "Ironworks"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Ironworks RENAME COLUMN Coefficient TO Production;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Ironworks SET Population=10 WHERE Level=1;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Ironworks SET Production=2400 WHERE Level=30;'''
cursor.execute(query)
conn.commit()

In [135]:
df = pd.read_sql_query("SELECT * FROM Ironworks", conn)
df

,Level,Wood,Clay,Iron,Population,Production,Building_id
0,1,75,65,70,10,30.0,Ironworks
1,2,93,82,86,2,39.0,Ironworks
2,3,117,105,107,2,50.0,Ironworks
3,4,147,134,132,3,61.0,Ironworks
4,5,184,171,164,3,74.0,Ironworks
5,6,231,219,203,4,88.0,Ironworks
6,7,289,279,251,4,103.0,Ironworks
7,8,362,356,310,5,120.0,Ironworks
8,9,453,454,384,6,138.0,Ironworks
9,10,568,579,476,7,159.0,Ironworks


### Granary

In [136]:
for level in range(1, 31):
    query = '''INSERT INTO Granary (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(45 * np.exp(0.262*(level-1)))
    clay = int(40 * np.exp(0.279*(level-1)))
    iron = int(30 * np.exp(0.255*(level-1)))
    population = 0
    coef = int(240 * np.exp(0.159*(level-1)))
    building_name = "Granary"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Granary RENAME COLUMN Coefficient TO Max_Population;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Granary SET Max_Population=24000 WHERE Level=30;'''
cursor.execute(query)
conn.commit()

In [146]:
df = pd.read_sql_query("SELECT * FROM Granary", conn)
df

,Level,Wood,Clay,Iron,Population,Max_Population,Building_id
0,1,45,40,30,0,240.0,Granary
1,2,58,52,38,0,281.0,Granary
2,3,75,69,49,0,329.0,Granary
3,4,98,92,64,0,386.0,Granary
4,5,128,122,83,0,453.0,Granary
5,6,166,161,107,0,531.0,Granary
6,7,216,213,138,0,623.0,Granary
7,8,281,281,178,0,730.0,Granary
8,9,366,372,230,0,856.0,Granary
9,10,475,492,297,0,1003.0,Granary


### Warehouse

In [147]:
for level in range(1, 31):
    query = '''INSERT INTO Warehouse (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(60 * np.exp(0.234*(level-1)))
    clay = int(50 * np.exp(0.241*(level-1)))
    iron = int(40 * np.exp(0.221*(level-1)))
    population = 0
    coef = int(1000 * np.exp(0.207*(level-1)))
    building_name = "Warehouse"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Warehouse RENAME COLUMN Coefficient TO Capacity;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Warehouse SET Capacity=400000 WHERE Level=30;'''
cursor.execute(query)
conn.commit()

In [151]:
df = pd.read_sql_query("SELECT * FROM Warehouse", conn)
df

,Level,Wood,Clay,Iron,Population,Capacity,Building_id
0,1,60,50,40,0,1000.0,Warehouse
1,2,75,63,49,0,1229.0,Warehouse
2,3,95,80,62,0,1512.0,Warehouse
3,4,121,103,77,0,1860.0,Warehouse
4,5,152,131,96,0,2288.0,Warehouse
5,6,193,166,120,0,2815.0,Warehouse
6,7,244,212,150,0,3462.0,Warehouse
7,8,308,270,187,0,4258.0,Warehouse
8,9,390,343,234,0,5238.0,Warehouse
9,10,492,437,292,0,6443.0,Warehouse


### Hideout

In [152]:
for level in range(1, 16):
    query = '''INSERT INTO Hideout (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(50 * np.exp(0.221*(level-1)))
    clay = int(60 * np.exp(0.221*(level-1)))
    iron = int(50 * np.exp(0.221*(level-1)))
    population = np.ceil(level/5)
    coef = coef = int(150 * np.exp(0.264*(level-1)))
    building_name = "Hideout"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Hideout RENAME COLUMN Coefficient TO Hidden_Resources;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Hideout SET Hidden_Resources=6000 WHERE Level=15;'''
cursor.execute(query)
conn.commit()

In [163]:
df = pd.read_sql_query("SELECT * FROM Hideout", conn)
df

,Level,Wood,Clay,Iron,Population,Hidden_Resources,Building_id
0,1,50,60,50,1,150.0,Hideout
1,2,62,74,62,1,195.0,Hideout
2,3,77,93,77,1,254.0,Hideout
3,4,97,116,97,1,331.0,Hideout
4,5,121,145,121,1,431.0,Hideout
5,6,150,181,150,2,561.0,Hideout
6,7,188,225,188,2,731.0,Hideout
7,8,234,281,234,2,952.0,Hideout
8,9,292,351,292,2,1239.0,Hideout
9,10,365,438,365,2,1614.0,Hideout


### Wall

In [ ]:
prev_coef = 0
for level in range(1, 21):
    query = '''INSERT INTO Wall (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(50 * np.exp(0.235*(level-1)))
    clay = int(100 * np.exp(0.25*(level-1)))
    iron = int(20 * np.exp(0.24*(level-1)))
    population = int(np.exp(0.146*(level-1)))
    coef = prev_coef + int(5 + np.floor(level/5))
    building_name = "Wall"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Wall RENAME COLUMN Coefficient TO Coefficient_Defence;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Wall SET Population=5 WHERE Level=1'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Wall SET Coefficient_Defence=135 WHERE Level=20'''
cursor.execute(query)
conn.commit()

In [190]:
df = pd.read_sql_query("SELECT * FROM Wall", conn)
df

,Level,Wood,Clay,Iron,Population,Coefficient_Defence,Building_id
0,1,50,100,20,5,5.0,Wall
1,2,63,128,25,1,10.0,Wall
2,3,79,164,32,1,15.0,Wall
3,4,101,211,41,1,20.0,Wall
4,5,127,271,52,1,26.0,Wall
5,6,161,349,66,2,32.0,Wall
6,7,204,448,84,2,38.0,Wall
7,8,259,575,107,2,44.0,Wall
8,9,327,738,136,3,50.0,Wall
9,10,414,948,173,3,57.0,Wall


### Church

In [199]:
query = '''INSERT INTO Church (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
level = 5
wood = 20000
clay = 28000
iron = 12000
population = 2700
coef = 10
building_name = "Church"
cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
conn.commit()

cursor.execute("ALTER TABLE Church RENAME Coefficient to Range;")
conn.commit()

In [255]:
df = pd.read_sql_query("SELECT * FROM Church", conn)
df

,Level,Wood,Clay,Iron,Population,Range,Building_id
0,1,100,200,50,100,1.0,Church
1,2,1500,3000,500,400,2.0,Church
2,3,5000,8000,1500,1000,4.0,Church
3,4,9000,14000,2500,1500,6.0,Church
4,5,20000,28000,12000,2500,10.0,Church


### Watchtower

In [243]:
for level in range(1, 16):
    query = '''INSERT INTO Watchtower (Level, Wood, Clay, Iron, Population, Coefficient, Building_id) VALUES (?,?,?,?,?,?,?)'''
    wood = int(7000 * np.exp(0.18*(level-1)))
    clay = int(10000 * np.exp(0.174*(level-1)))
    iron = int(6500 * np.exp(0.185*(level-1)))
    population = int(100*np.exp(0.146*(level-1)))
    coef = round(np.exp(0.1935*(level-1)), 1)
    building_name = "Watchtower"
    cursor.execute(query, (level, wood, clay, iron, population, coef, building_name))
    conn.commit()

query = '''ALTER TABLE Watchtower RENAME COLUMN Coefficient TO View_Range;'''
cursor.execute(query)
conn.commit()

query = '''UPDATE Watchtower SET Population=300 WHERE Level=1;'''
cursor.execute(query)
conn.commit()

In [252]:
df = pd.read_sql_query("SELECT * FROM Watchtower", conn)
df

,Level,Wood,Clay,Iron,Population,View_Range,Building_id
0,1,7000,10000,6500,300,1.0,Watchtower
1,2,8380,11900,7820,115,1.2,Watchtower
2,3,10033,14162,9410,133,1.5,Watchtower
3,4,12012,16853,11322,154,1.8,Watchtower
4,5,14381,20057,13623,179,2.2,Watchtower
5,6,17217,23869,16392,207,2.6,Watchtower
6,7,20612,28405,19723,240,3.2,Watchtower
7,8,24677,33804,23731,277,3.9,Watchtower
8,9,29544,40228,28554,321,4.7,Watchtower
9,10,35371,47874,34356,372,5.7,Watchtower


## Dupa

In [13]:
conn.close()

# 7	Palace
# 8	Sawmill
# 9	Brickyard
# 10	Ironworks
# 11	Granary
# 12	Warehouse
# 13	Hideout
# 14	Wall
# 15	Church
# 16	Watchtower

# SELECT sql FROM sqlite_master WHERE type='table' AND name='Buildings';